In [1]:
#import pandas_datareader.data as web
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import datetime
import yfinance as yf
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go


colors = {
    'background': '#571421',
    'text': '#f5f6fa'
}


app = dash.Dash()
app.title = "Stock Visualization"
app.layout = html.Div( children=
    [
        html.H2('Day Trading Stock Visualization Dashboard', style = {'textAlign': 'center','color': '#fafafa', 
                                                          'background-image': 'url(https://cdn.smartbrief.com/sites/default/files/styles/originals_full_summary_banner_style/public/original-content/AdobeStock_295042621.jpeg?itok=n6VoXVrA.png)'}),
        html.P('This dashboard provides you information that is essential for day trading purposes. Dashboard shows moving average, sentiment analysis, and performance of the selected stock against the past three days.', style = {'textAlign': 'left','color': '#0c0d0d'}),
        html.Br(),
        dcc.Input(id="input", type="text", placeholder="symbol", style={'marginRight':'10px'}),
        dcc.Input(id="input1", type="text", placeholder="Period:1d, 2d and so on", debounce=True),
        dcc.Input(id="input2", type="text", placeholder="Interval: 1m, 2m, 5m", debounce=True),
        dcc.Input(id="input3", type="number", placeholder="Moving Avarage", debounce=True),
        dcc.Dropdown(id='demo-dropdown', style={'backgroundColor': '#fafbfc'}, options=[{'label': 'Yesterday', 'value': 4},
                                                 {'label': 'the day before yesterday', 'value': 3},{'label': 'two days ago', 'value': 2}],
                     value='select past 3 days stock performance', placeholder="Select the date you want to compare with"),
        
        html.Div(id="output-graph"),
    ]
)



@app.callback(
    Output(component_id="output-graph", component_property='children'),
    [Input(component_id="input", component_property="value")],
     [Input(component_id="input1", component_property="value")],
      [Input(component_id="input2", component_property="value")],
      [Input(component_id="input3", component_property="value")],
    [Input(component_id="demo-dropdown", component_property="value")],


    
 
)
def update_value(input_data, period1, interval1, MA, date):
    
    stocks = (yf.Ticker(input_data))
    stock=stocks.history(period= period1, interval = interval1).reset_index()
    stock2=stocks.history(period= '5d', interval = '5m')#.reset_index()
    stock3=stocks.history(period= '5d', interval = '5m').reset_index()
    stock5=stocks.history(period= '5d', interval = '1d')#.reset_index()
    data=yf.Ticker(input_data).history(period= "10d", interval = "1d").reset_index()
    
    stock5.index = stock5.index.astype(str) # convert the format of the string to string (from date time format to string format)
    b1 =stock5.reset_index().values # convert the dataframe to values
    time_day = (b1[:,0]).tolist() # only take the first column and convert it to list

    stock2.index = stock2.index.astype(str) # convert the format of the string to string (from date time format to string format)
    b =stock2.reset_index().values # convert the dataframe to values
    time = (b[:,0]).tolist() # only take the first column and convert it to list

    stock3['date'] = time
    stock4=stock3.set_index('date')
    
    import subm
    dff = subm.data(input_data)
    dff
    
    short_rolling = stock['Close'].rolling(window=MA).mean()
    short_rolling2 = stock4['Close'].rolling(window=MA).mean()
    stock['mv']= short_rolling
    stock4['mv'] = short_rolling2
  
    fig = make_subplots(rows=2, cols=2, subplot_titles=(input_data + ' ' + time_day[4] + ' '+ ' Realtime Price','Stock Price Movement for selected stock' , input_data + ' ' + time_day[date-1]+ ' '+ ' Past Days Performance' , 'Sentiment Analysis - Compound values for stock selected'),  vertical_spacing=0.25)
    fig = fig.add_trace(go.Scatter(x=stock['Datetime'], y=stock["Open"]),row=1, col=1)
    fig = fig.add_trace(go.Candlestick(x=stock['Datetime'],
                                        open=stock['Open'],
                                        high=stock['High'],
                                        low=stock['Low'],
                                        close=stock['Close']),
                         row=1, col=1)
    fig = fig.add_trace(go.Candlestick(x=stock4['Datetime'].loc[time_day[date-1] :time_day[date]],
                                        open=stock4['Open'].loc[time_day[date-1] :time_day[date]],
                                        high=stock4['High'].loc[time_day[date-1] :time_day[date]],
                                        low=stock4['Low'].loc[time_day[date-1] :time_day[date]],
                                        close=stock4['Close'].loc[time_day[date-1] :time_day[date]]),
                         row=2, col=1)#.update_layout(xaxis_rangeslider_visible=False, width=1600,height=800)
    fig = fig.add_trace(go.Scatter(x=stock4['Datetime'].loc[time_day[date-1] :time_day[date]], y=stock4['mv'].loc[time_day[date-1] :time_day[date]]),row=2, col=1)
    fig = fig.add_trace(go.Scatter(x=stock['Datetime'],  y=stock['mv']),row=1, col=1)
    fig = fig.add_trace(go.Scatter(x=data['Date'], y=data["Close"]),row=1, col=2)
    fig.add_trace(go.Bar(x=dff['date'], y=dff[input_data]),row = 2, col = 2)
    
    
    fig.update_layout(xaxis_rangeslider_visible=False, width=1500,height=800)
    # fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)'})
 

    
    
    return dcc.Graph(id="demo" , figure=fig)#.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)'})

    
   
    
    
    
if __name__ == "__main__":
    app.run_server(debug=False, port = 8383)

Dash is running on http://127.0.0.1:8383/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8383/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Aug/2021 18:48:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2021 18:48:42] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2021 18:48:42] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2021 18:48:42] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\sumit\

127.0.0.1 - - [03/Aug/2021 18:48:43] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\sumit\

...

127.0.0.1 - - [03/Aug/2021 18:48:48] "POST /_dash-update-component HTTP/1.1" 500 -
c:\Users\sumit\Desktop\fintech-Project-3\Zeray\subm.py:23: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 23 of the file c:\Users\sumit\Desktop\fintech-Project-3\Zeray\subm.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  html = BeautifulSoup(response)
c:\Users\sumit\Desktop\fintech-Project-3\Zeray\subm.py:23: GuessedAtParserWarning:

No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\sumit\

127.0.0.1 - - [03/Aug/2021 18:48:56] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\sumit\anaconda3\envs\project3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\sumit\

127.0.0.1 - - [03/Aug/2021 18:48:56] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [03/Aug/2021 18:48:57] "POST /_dash-update-component HTTP/1.1" 200 -
